In [1]:
import CeLEry as cel

import os,csv,re
import pandas as pd
import numpy as np
import scanpy as sc
import math
from skimage import io, color

from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from anndata import AnnData, read_h5ad
import seaborn as sns
from tqdm import tqdm
import matplotlib.ticker as mtick
import scipy

import json

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_merfish_raw = pd.read_csv("data/datasets_mouse_brain_map_BrainReceptorShowcase_Slice1_Replicate1_cell_by_gene_S1R1.csv", index_col=0)   
meta_data = pd.read_csv("data/datasets_mouse_brain_map_BrainReceptorShowcase_Slice1_Replicate1_cell_metadata_S1R1.csv", index_col=0)

data_merfish = AnnData(data_merfish_raw)

data_merfish.obs['x_cord'] = meta_data['center_x'].tolist()
data_merfish.obs['y_cord'] = meta_data['center_y'].tolist()
data_merfish_raw = data_merfish.copy()

sc.pp.filter_cells(data_merfish, min_counts=500)
sc.pp.filter_cells(data_merfish, min_genes=100)

sc.pp.neighbors(data_merfish, n_neighbors=15, use_rep='X', random_state=1)
sc.tl.louvain(data_merfish, 0.4, random_state=1)

In [3]:
prefix = "output/brain/"
celery_pred_right = pd.read_csv(prefix + "right_brain_celery.csv", index_col=0)
celery_pred_left = pd.read_csv(prefix + "left_brain_celery.csv", index_col=0)
celery_pred = pd.concat([celery_pred_right, celery_pred_left], axis=0)

tangram_pred_right = pd.read_csv(prefix + "right_brain_tangram.csv", index_col=0)
tangram_pred_left = pd.read_csv(prefix + "left_brain_tangram.csv", index_col=0)
tangram_pred = pd.concat([tangram_pred_right, tangram_pred_left], axis=0)

spaotsc_pred_right = pd.read_csv(prefix + "right_brain_spaotsc.csv", index_col=0)
spaotsc_pred_left = pd.read_csv(prefix + "left_brain_spaotsc.csv", index_col=0)
spaotsc_pred = pd.concat([spaotsc_pred_right, spaotsc_pred_left], axis=0)

novosparc_pred_right = pd.read_csv(prefix + "right_brain_novosparc.csv", index_col=0)
novosparc_pred_left = pd.read_csv(prefix + "left_brain_novosparc.csv", index_col=0)
novosparc_pred = pd.concat([novosparc_pred_right, novosparc_pred_left], axis=0)

In [4]:
data_merfish.obs = data_merfish.obs.join(celery_pred['x'])
data_merfish.obs = data_merfish.obs.join(celery_pred['y'])
data_merfish.obs = data_merfish.obs.rename(columns={"x": "x_celery", "y": "y_celery"})

data_merfish.obs = data_merfish.obs.join(tangram_pred['x'])
data_merfish.obs = data_merfish.obs.join(tangram_pred['y'])
data_merfish.obs = data_merfish.obs.rename(columns={"x": "x_tangram", "y": "y_tangram"})

data_merfish.obs = data_merfish.obs.join(spaotsc_pred['x'])
data_merfish.obs = data_merfish.obs.join(spaotsc_pred['y'])
data_merfish.obs = data_merfish.obs.rename(columns={"x": "x_spaotsc", "y": "y_spaotsc"})

data_merfish.obs = data_merfish.obs.join(novosparc_pred['x'])
data_merfish.obs = data_merfish.obs.join(novosparc_pred['y'])
data_merfish.obs = data_merfish.obs.rename(columns={"x": "x_novosparc", "y": "y_novosparc"})

In [5]:
def normalize(true, predict):
    truex = true[:, 0]
    truey = true[:, 1]
    predictx = predict[:, 0]
    predicty = predict[:, 1]

    truexmin = np.min(truex) - 1
    truexmax = np.max(truex) + 1
    trueymin = np.min(truey) - 1
    trueymax = np.max(truey) - 1

    predictxmin = np.min(predictx) - 1
    predictxmax = np.max(predictx) + 1
    predictymin = np.min(predicty) - 1
    predictymax = np.max(predicty) - 1

    relativex = (predictx - predictxmin)/(predictxmax - predictxmin)
    relativey = (predicty - predictymin)/(predictymax - predictymin)

    scalex = relativex * (truexmax - truexmin) + truexmin
    scaley = relativey * (trueymax - trueymin) + trueymin
    return np.array([scalex, scaley])

leftdata = data_merfish[data_merfish.obs['x_cord'] <= np.quantile(data_merfish.obs['x_cord'], 0.5), :]
rightdata = data_merfish[data_merfish.obs['x_cord'] > np.quantile(data_merfish.obs['x_cord'], 0.5), :]

leftnormdata = normalize(np.array(leftdata.obs[['x_cord', 'y_cord']]), np.array(leftdata.obs[['x_spaotsc', 'y_spaotsc']])).T
rightnormdata = normalize(np.array(rightdata.obs[['x_cord', 'y_cord']]), np.array(rightdata.obs[['x_spaotsc', 'y_spaotsc']])).T
leftnormdata = pd.DataFrame(leftnormdata, index=leftdata.obs.index, columns=['x_spaotscnorm', 'y_spaotscnorm'])
rightnormdata = pd.DataFrame(rightnormdata, index=rightdata.obs.index, columns=['x_spaotscnorm', 'y_spaotscnorm'])
normdata = pd.concat([leftnormdata, rightnormdata], axis=0)
data_merfish.obs = data_merfish.obs.join(normdata)
del normdata, leftnormdata, rightnormdata

leftnormdata = normalize(np.array(leftdata.obs[['x_cord', 'y_cord']]), np.array(leftdata.obs[['x_tangram', 'y_tangram']])).T
rightnormdata = normalize(np.array(rightdata.obs[['x_cord', 'y_cord']]), np.array(rightdata.obs[['x_tangram', 'y_tangram']])).T
leftnormdata = pd.DataFrame(leftnormdata, index=leftdata.obs.index, columns=['x_tangramnorm', 'y_tangramnorm'])
rightnormdata = pd.DataFrame(rightnormdata, index=rightdata.obs.index, columns=['x_tangramnorm', 'y_tangramnorm'])
normdata = pd.concat([leftnormdata, rightnormdata], axis=0)
data_merfish.obs = data_merfish.obs.join(normdata)
del normdata, leftnormdata, rightnormdata

leftnormdata = normalize(np.array(leftdata.obs[['x_cord', 'y_cord']]), np.array(leftdata.obs[['x_novosparc', 'y_novosparc']])).T
rightnormdata = normalize(np.array(rightdata.obs[['x_cord', 'y_cord']]), np.array(rightdata.obs[['x_novosparc', 'y_novosparc']])).T
leftnormdata = pd.DataFrame(leftnormdata, index=leftdata.obs.index, columns=['x_novosparcnorm', 'y_novosparcnorm'])
rightnormdata = pd.DataFrame(rightnormdata, index=rightdata.obs.index, columns=['x_novosparcnorm', 'y_novosparcnorm'])
normdata = pd.concat([leftnormdata, rightnormdata], axis=0)
data_merfish.obs = data_merfish.obs.join(normdata)
del normdata, leftnormdata, rightnormdata

Spatial prediction plot colored by cell type

In [11]:
x_min = np.min([np.min(data_merfish.obs['x_cord']), np.min(data_merfish.obs['x_celery']), np.min(data_merfish.obs['x_tangram']), np.min(data_merfish.obs['x_spaotsc']), np.min(data_merfish.obs['x_novosparc'])]) - 150
y_min = np.min([np.min(data_merfish.obs['y_cord']), np.min(data_merfish.obs['y_celery']), np.min(data_merfish.obs['y_tangram']), np.min(data_merfish.obs['y_spaotsc']), np.min(data_merfish.obs['y_novosparc'])]) - 150
x_max = np.max([np.max(data_merfish.obs['x_cord']), np.max(data_merfish.obs['x_celery']), np.max(data_merfish.obs['x_tangram']), np.max(data_merfish.obs['x_spaotsc']), np.max(data_merfish.obs['x_novosparc'])]) + 150
y_max = np.max([np.max(data_merfish.obs['y_cord']), np.max(data_merfish.obs['y_celery']), np.max(data_merfish.obs['y_tangram']), np.max(data_merfish.obs['y_spaotsc']), np.max(data_merfish.obs['y_novosparc'])]) + 150


colors = {0: '#FDDDE6', 1: '#FCE883', 2: '#B4674D', 3: '#EE204D', 4: '#FF7538', 5: '#1CAC78', 6: '#E7C697', 7: '#1F75FE', 8: '#6E5160',
    9: '#CDC5C2', 10: '#FE4EDA', 11: '#DD9475', 12: '#000000', 13:'#95918C', 14: '#9F8170', 15: '#FFA343', 16: '#80DAEB', 17: '#17806D'} 
colors = {str(key): value for key, value in colors.items()}

fig, axes = plt.subplots(1, 5, figsize=(30, 5))
sns.scatterplot(data=data_merfish.obs, x="x_cord", y="y_cord",s=4, hue="louvain", palette=colors, 
                ax=axes[0], legend=False).set(title="Truth", xlabel=None, ylabel=None, xlim=(x_min, x_max), ylim=(y_min, y_max))
sns.scatterplot(data=data_merfish.obs, x="x_celery", y="y_celery",s=4, hue="louvain", palette=colors, 
                ax=axes[1], legend=False).set(title="CeLEry", xlabel=None, ylabel=None, xlim=(x_min, x_max), ylim=(y_min, y_max))
sns.scatterplot(data=data_merfish.obs, x="x_tangramnorm", y="y_tangramnorm",s=4, hue="louvain", palette=colors, 
                ax=axes[2], legend=False).set(title="Tangram", xlabel=None, ylabel=None, xlim=(x_min, x_max), ylim=(y_min, y_max))
sns.scatterplot(data=data_merfish.obs, x="x_spaotscnorm", y="y_spaotscnorm",s=4, hue="louvain", palette=colors, 
                ax=axes[3], legend=False).set(title="SpaOTsc", xlabel=None, ylabel=None, xlim=(x_min, x_max), ylim=(y_min, y_max))
sns.scatterplot(data=data_merfish.obs, x="x_novosparcnorm", y="y_novosparcnorm",s=4, hue="louvain", palette=colors, 
                ax=axes[4], legend=True).set(title="novoSpaRc", xlabel=None, ylabel=None, xlim=(x_min, x_max), ylim=(y_min, y_max))
fig.subplots_adjust(right=0.85)
plt.legend(title="Cluster", loc="upper right", bbox_to_anchor=(1.35, 0.95))

figname = "output/brain/plot/type_pred.pdf"
fig.savefig(figname)
plt.close(fig)

Spatial plot colored by Euclidian distance between true location and predicted location

In [12]:
pred_dist_celery = np.sqrt(np.sum(sq(np.array(data_train.obs[['x_cord', 'y_cord']]), np.array(data_train.obs[['x_celery', 'y_celery']])), axis=1))
pred_dist_tangram = np.sqrt(np.sum(sq(np.array(data_train.obs[['x_cord', 'y_cord']]), np.array(data_train.obs[['x_tangramnorm', 'y_tangramnorm']])), axis=1))
pred_dist_spaotsc = np.sqrt(np.sum(sq(np.array(data_train.obs[['x_cord', 'y_cord']]), np.array(data_train.obs[['x_spaotscnorm', 'y_spaotscnorm']])), axis=1))
pred_dist_novosparc = np.sqrt(np.sum(sq(np.array(data_train.obs[['x_cord', 'y_cord']]), np.array(data_train.obs[['x_novosparcnorm', 'y_novosparcnorm']])), axis=1))
vmax = np.max([pred_dist_celery, pred_dist_tangram])

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4,figsize=(20,5))
ax1.scatter(data_train.obs['x_cord'], data_train.obs['y_cord'], s=1, c=pred_dist_celery, vmin=0, vmax=vmax)
ax1.set_title('CeLEry')
ax2.scatter(data_train.obs['x_cord'], data_train.obs['y_cord'], s=1, c=pred_dist_tangram, vmin=0, vmax=vmax)
ax2.set_title('Tangram')
ax3.scatter(data_train.obs['x_cord'], data_train.obs['y_cord'], s=1, c=pred_dist_spaotsc, vmin=0, vmax=vmax)
ax3.set_title('SpaOTsc')
a1 = ax4.scatter(data_train.obs['x_cord'], data_train.obs['y_cord'], s=1, c=pred_dist_novosparc, vmin=0, vmax=vmax)
ax4.set_title('novoSpaRc')

fig.colorbar(a1, fraction=0.046, pad=0.04)
figname = "output/brain/plot/pred_error.pdf"
fig.savefig(figname)
plt.close(fig)

Correlation between true X/Y value and predicted X/Y value

In [14]:
x_corr = [scipy.stats.pearsonr(data_merfish.obs['x_cord'], data_merfish.obs['x_celery']).statistic, scipy.stats.pearsonr(data_merfish.obs['x_cord'], data_merfish.obs['x_tangram']).statistic,
            scipy.stats.pearsonr(data_merfish.obs['x_cord'], data_merfish.obs['x_spaotsc']).statistic, scipy.stats.pearsonr(data_merfish.obs['x_cord'], data_merfish.obs['x_novosparc']).statistic]

y_corr = [scipy.stats.pearsonr(data_merfish.obs['y_cord'], data_merfish.obs['y_celery']).statistic, scipy.stats.pearsonr(data_merfish.obs['y_cord'], data_merfish.obs['y_tangram']).statistic,
            scipy.stats.pearsonr(data_merfish.obs['y_cord'], data_merfish.obs['y_spaotsc']).statistic, scipy.stats.pearsonr(data_merfish.obs['y_cord'], data_merfish.obs['y_novosparc']).statistic]

value = [*x_corr, *y_corr]
method = ['CeLEry', 'Tangram', 'SpaOTsc', 'novoSpaRc']*2
all = np.repeat(['X axis', 'Y axis'], 4)
rmse_df = pd.DataFrame(np.array([value, method, all])).T
rmse_df.columns = ['value', 'method', 'all']
rmse_df.value = rmse_df.value.astype('float')
cols = ['#CAE7B9', '#F3DE8A', '#EB9486', '#7E7F9A']

fig, axes = plt.subplots(1, 1, figsize=(6, 5))
sns.barplot(data=rmse_df, x='all',  y='value', hue="method",  palette=cols)
plt.legend(bbox_to_anchor=(1.05, 0.3), loc=3, borderaxespad=0,fontsize=10)
plt.ylabel('Correlation', {'size' :10})
plt.xlabel('')
fig.subplots_adjust(right=0.75)
plt.grid(axis='both', color='silver', alpha=0.3)

ax = plt.gca()
ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2f'))
ax.grid(True)
ax.set_xmargin(0.05)

figname = "output/brain/plot/pred_corr.pdf"
fig.savefig(figname)
plt.close(fig)

Pairwise distance calculation and pairwise distance density plot

In [15]:
def distCompute(data_merfish):
    celery_dist = []
    tangram_dist = []
    true_dist = []
    spaotsc_dist = []
    novosparc_dist = []
    Qdata_loc = np.array(data_merfish.obs[['x_cord', 'y_cord']])
    celery_pred = np.array(data_merfish.obs[['x_celery', 'y_celery']])
    tangram_pred = np.array(data_merfish.obs[['x_tangramnorm', 'y_tangramnorm']])
    spaotsc_pred = np.array(data_merfish.obs[['x_spaotscnorm', 'y_spaotscnorm']])
    novosparc_pred = np.array(data_merfish.obs[['x_novosparcnorm', 'y_novosparcnorm']])

    for i in tqdm(range(Qdata_loc.shape[0])):
        # for j in range(i+1, Qdata_loc.shape[0]):
        celery_i = celery_pred[i, :]
        celery_points = celery_pred[i+1:, :]
        celery_dist.extend(np.sqrt(np.sum((celery_points - celery_i)**2, axis=1)))

        tangram_i = tangram_pred[i, :]
        tangram_points = tangram_pred[i+1:, :]
        tangram_dist.extend(np.sqrt(np.sum((tangram_points - tangram_i)**2, axis=1)))

        spaotsc_i = spaotsc_pred[i, :]
        spaotsc_points = spaotsc_pred[i+1:, :]
        spaotsc_dist.extend(np.sqrt(np.sum((spaotsc_points - spaotsc_i)**2, axis=1)))

        novosparc_i = novosparc_pred[i, :]
        novosparc_points = novosparc_pred[i+1:, :]
        novosparc_dist.extend(np.sqrt(np.sum((novosparc_points - novosparc_i)**2, axis=1)))

        true_i = Qdata_loc[i, :]
        true_points = Qdata_loc[i+1:, :]
        true_dist.extend(np.sqrt(np.sum((true_points - true_i)**2, axis=1)))
    return celery_dist, tangram_dist, spaotsc_dist, novosparc_dist, true_dist

celery_dist, tangram_dist, spaotsc_dist, novosparc_dist, true_dist = distCompute(data_merfish[data_merfish.obs['x_cord'] <= np.quantile(data_merfish.obs['x_cord'], 0.5)])
celery_dist_r, tangram_dist_r, spaotsc_dist_r, novosparc_dist_r, true_dist_r = distCompute(data_merfish[data_merfish.obs['x_cord'] > np.quantile(data_merfish.obs['x_cord'], 0.5)])

100%|██████████| 9171/9171 [00:13<00:00, 681.64it/s] 


In [16]:
celery_dist.extend(celery_dist_r)
tangram_dist.extend(tangram_dist_r)
spaotsc_dist.extend(spaotsc_dist_r)
novosparc_dist.extend(novosparc_dist_r)
true_dist.extend(true_dist_r)

In [17]:
value = [scipy.stats.pearsonr(true_dist, celery_dist).statistic, scipy.stats.pearsonr(true_dist, tangram_dist).statistic,
            scipy.stats.pearsonr(true_dist, spaotsc_dist).statistic, scipy.stats.pearsonr(true_dist, novosparc_dist).statistic]
method = ['CeLEry', 'Tangram', 'SpaOTsc', 'novoSpaRc']
all = np.repeat('Method', 4)
rmse_df = pd.DataFrame(np.array([value, method, all])).T
rmse_df.columns = ['value', 'method', 'all']
rmse_df.value = rmse_df.value.astype('float')
cols = ['#CAE7B9', '#F3DE8A', '#EB9486', '#7E7F9A']

fig, axes = plt.subplots(1, 1, figsize=(6, 5))
corr_plot = sns.barplot(data=rmse_df,  y='value', x="method",  palette=cols)
corr_plot.set(ylim=(0, 1))
plt.ylabel('Pairwise Distance Correlation', {'size' :10})
plt.xlabel('Methods')


plt.grid(axis='both', color='silver', alpha=0.3)

ax = plt.gca()
ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2f'))
ax.grid(True)
ax.set_xmargin(0.05)

os.chdir("/Users/shunzhou/Desktop/RA/CeLEry-main/output/brain")
figname = "output/brain/plot/pairwise_corr.pdf"
fig.savefig(figname)
plt.close(fig)

In [19]:
import mpl_scatter_density # adds projection='scatter_density'
from matplotlib.colors import LinearSegmentedColormap

white_viridis = LinearSegmentedColormap.from_list('white_viridis', [
    (0, '#ffffff'),
    (1e-20, '#440053'),
    (0.2, '#404388'),
    (0.4, '#2a788e'),
    (0.6, '#21a784'),
    (0.8, '#78d151'),
    (1, '#fde624'),
], N=256)

max_lim = np.max([np.max(celery_dist)])


def using_mpl_scatter_density(ax, x, y, title, label=True):
    density = ax.scatter_density(x, y, cmap=white_viridis)
    plt.title(title)
    if label:
        plt.ylabel("Pairwise distance between predicted coordinates", fontsize=15)
    lims = [0, max_lim]
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    plt.ylim((0, max_lim))
    plt.xlim((0, max_lim))
    fig.colorbar(density)
    
fig = plt.figure(figsize=(30,6))
ax1 = fig.add_subplot(1,4,1,projection='scatter_density')
using_mpl_scatter_density(ax1, true_dist, celery_dist, title="CeLEry")
ax2 = fig.add_subplot(1,4,2,projection='scatter_density')
using_mpl_scatter_density(ax2, true_dist, tangram_dist, title="Tangram", label=False)
ax3 = fig.add_subplot(1,4,3,projection='scatter_density')
using_mpl_scatter_density(ax3, true_dist, spaotsc_dist, title="SpaOTsc", label=False)
ax4 = fig.add_subplot(1,4,4,projection='scatter_density')
using_mpl_scatter_density(ax4, true_dist, novosparc_dist, title="novoSpaRc", label=False)
fig.text(0.5, 0.02, 'Pairwise distance between true coordinates', va='center', ha='center', fontsize=15)

figname = "output/brain/plot/pairwise.pdf"
fig.savefig(figname)
plt.close(fig)

Relative gene expression map

In [6]:
Qdata_df = pd.DataFrame(data_merfish.X.copy())
Qdata_df.columns = data_merfish.var.index

# gene_lst = Qdata_df.columns
gene_lst = ["Adrb1", "Cx3cl1", "Grm5"]

x_min = np.min([np.min(data_merfish.obs['x_cord']), np.min(data_merfish.obs['x_celery']), np.min(data_merfish.obs['x_tangram']), np.min(data_merfish.obs['x_spaotsc']), np.min(data_merfish.obs['x_novosparc'])]) - 150
x_max = np.max([np.max(data_merfish.obs['x_cord']), np.max(data_merfish.obs['x_celery']), np.max(data_merfish.obs['x_tangram']), np.min(data_merfish.obs['x_spaotsc']), np.min(data_merfish.obs['x_novosparc'])]) + 150
y_min = np.min([np.min(data_merfish.obs['y_cord']), np.min(data_merfish.obs['y_celery']), np.min(data_merfish.obs['y_tangram']), np.min(data_merfish.obs['y_spaotsc']), np.min(data_merfish.obs['y_novosparc'])]) - 150
y_max = np.max([np.max(data_merfish.obs['y_cord']), np.max(data_merfish.obs['y_celery']), np.max(data_merfish.obs['y_tangram']), np.min(data_merfish.obs['y_spaotsc']), np.min(data_merfish.obs['y_novosparc'])]) + 150

for i in gene_lst:
    map_col = "lightgoldenrodyellow"
    point_col = "GnBu"

    cmap_spa = Qdata_df[i]
    cmap_spa = np.stack(cmap_spa)
    
    cmap_spa = cmap_spa/np.max(cmap_spa)

    min_val = np.min([np.min(cmap_spa)])
    max_val = np.max([np.max(cmap_spa)])


    fig = plt.figure(figsize=(25,5))
    ax1 = fig.add_subplot(1,5,1)
    ax1.set_facecolor(map_col)
    ax1.scatter(data_merfish.obs['x_cord'], data_merfish.obs['y_cord'],s=1,c=cmap_spa, cmap=plt.get_cmap(point_col), vmin=min_val, vmax=max_val)
    ax1.set_xlim(x_min, x_max)
    ax1.set_title("Truth")

    ax2 = fig.add_subplot(1,5,2)
    ax2.set_facecolor(map_col)
    ax2.scatter(data_merfish.obs['x_celery'], data_merfish.obs['y_celery'], s=1,c=cmap_spa, cmap=plt.get_cmap(point_col), vmin=min_val, vmax=max_val)
    ax2.set_xlim(x_min, x_max)
    ax2.set_title("CeLEry")

    ax3 = fig.add_subplot(1,5,3)
    ax3.set_facecolor(map_col)
    ax3.scatter(data_merfish.obs['x_tangramnorm'], data_merfish.obs['y_tangramnorm'], s=1,c=cmap_spa, cmap=plt.get_cmap(point_col), vmin=min_val, vmax=max_val)
    ax3.set_xlim(x_min, x_max)
    ax3.set_title("Tangram")

    ax4 = fig.add_subplot(1,5,4)
    ax4.set_facecolor(map_col)
    ax4.scatter(data_merfish.obs['x_spaotscnorm'], data_merfish.obs['y_spaotscnorm'], s=1,c=cmap_spa, cmap=plt.get_cmap(point_col), vmin=min_val, vmax=max_val)
    ax4.set_xlim(x_min, x_max)
    ax4.set_title("SpaOTsc")

    ax5 = fig.add_subplot(1,5,5)
    a5 = ax5.set_facecolor(map_col)
    ax5.scatter(data_merfish.obs['x_novosparcnorm'], data_merfish.obs['y_novosparcnorm'], s=1,c=cmap_spa, cmap=plt.get_cmap(point_col), vmin=min_val, vmax=max_val)
    ax5.set_xlim(x_min, x_max)
    ax5.set_title("novoSpaRc")

    colormap = plt.cm.get_cmap(point_col)
    sm = plt.cm.ScalarMappable(cmap=colormap)
    fig.colorbar(sm, fraction=0.046, pad=0.04)
    fig.text(0.07, 0.5, i, va='center', ha='center', rotation='vertical', fontsize = 12)


    figname = "output/brain/plot/" + i +".pdf"
    fig.savefig(figname)
    plt.close(fig)